In [ ]:
# urlib 임포트
from urllib.request import urlopen
from bs4 import BeautifulSoup

# https://news.daum.net/
req = urlopen("https://news.daum.net/")
# print("응답 코드:", req.getcode())  # 200 : 성공

if req.getcode() == 200:
    # 성공
    # html 받아오기
    html = req.read()
    html = html.decode("utf-8")
    print("SUCCESS")
else:
    # 실패
    print("HTTP-ERROR")

In [ ]:
# text -> html dom으로 변경
soup = BeautifulSoup(html, "html.parser")
# print(soup.prettify())
# 문서 정보의 확인
print("문서 제목:", soup.title.text)

In [ ]:
# 주요 기사 div를 검색
# select or find
articles = soup.select("#mArticle")[0]  # id=mArticle
# print(type(articles))  # 여러 개 있음.. index가 0번인것부터 봐보자.
# print(articles.prettify())
# div#mArticle 자손인 a.link_txt를 검색
links = articles.select("a.link_txt")  # class=link_txt인 a 태그
# print(links)  # 여러 개 있음..
# print(links[0])
# 기사 목록을 { "link": ..., "title":...} 의 리스트

news_list = []

for link in links:
#     print(link)
    news = {
        "link": link['href'],  # 속성에 접근
        "title": link.text.strip()
    }
    news_list.append(news)
# print(news_list)

print("다음 뉴스 주요 기사:")
for news in news_list:
    print("{} - {}".format(news.get('title'), news.get('link')))

In [ ]:
# 연습
# 다음 뉴스 상단 div#cSub인 뉴스 수집

In [ ]:
# 네이버 영화
# https://movie.naver.com/movie/running/current.nhn
# 현재 상영영화 순위를 수집
req = urlopen("https://movie.naver.com/movie/running/current.nhn")
html_str = req.read().decode('utf-8')
# print(html_str[:256])  # html text

In [ ]:
# DOM 변환
soup = BeautifulSoup(html_str, "html.parser")
# print(soup.body)

In [ ]:
# ul.lst_detail_t1
current_movies = soup.select("ul.lst_detail_t1 > li") 
# class=lst_detail_t1인 ul의 자식인 li 태그들
# print(current_movies)

# 영화 세부 정보 검색
movie_list = []

for movie in current_movies: # li
    titles = movie.select("dl.lst_dsc > .tit > a")[0]
    
    info = {"title": titles.text.strip(), "link": titles['href']}
    movie_list.append(info)
    
# print(movie_list)
print("네이버 영화 순위")

for rank, info in enumerate(movie_list):
    print("{}위 - {} : {}".format(rank+1, info['title'], info['link']))

In [ ]:
# 이미지 다운로드
# ul.lst_detail_t1 > li -> img
# img 태그의 속성 src, alt(대체 텍스트)

import os
import re  # 정규표현식
from urllib.request import urlretrieve  # 네이버 영화 > 이미지 다운로드

if not os.path.exists("d:/data-science/crawl"):  # 디렉터리 없으면
    os.makedirs("d:/data-science/crawl")  # 생성

for movie in current_movies:
    img_tag = movie.select("img")[0]
#     print("포스터:", img_tag)
#     print("src:", img_tag['src'])
#     print("alt:", img_tag['alt'])
    # 파일명으로 사용할 수 없는 문자를 제거
    title = img_tag['alt']
    title = re.sub(r"[\/:*?<>|.]", "-",  title.strip())  # \/:*?<>|. 이런 문자들이 나오면 -로 바꿔줘라
#     print("정제된 제목:", title)
    src = img_tag['src']  # 이미지 주소
    target = "d:/data-science/crawl/{}.jpg".format(title)
    
    # 이미지 다운로드 : urlretrieve
    urlretrieve(src, target)
    print(".", end="")
else:
    print("Download Complete!")

In [ ]:
# 연습문제
# https://movie.naver.com/movie/running/premovie.nhn
# 개봉 예정 영화 정보를 수집해 봅시다.